In [21]:
import sqlite3
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

db_file = 'elderspeak_detect.db'
con = sqlite3.connect(db_file)
df = pd.read_sql_query('''
    select 	pr.spraaksnelheid,
            pr.geluidsniveau,
            pr.toonhoogte,
            tr.cilt,
            tr.woordlengteratio,
            tr.aantal_collectieve_voornaamwoorden,
            tr.aantal_verkleinwoorden,
            tr.aantal_herhalingen,
            tr.textcat_elderspeak_score
    from praat_resultaten pr
    join tekst_resultaten tr on tr.audio_id = pr.audio_id
    join teksten t on t.tekst_id = tr.tekst_id
    where t.methode = 'GOOGLE_ENKEL_NL_BE'
    and pr.spraaksnelheid > 0;''', con)
con.close()

print(f"Aantal kolommen: { len(df.columns) }")
print(f"Aantal rijen: { len(df.index) }")
df.tail()

Aantal kolommen: 9
Aantal rijen: 39


,spraaksnelheid,geluidsniveau,toonhoogte,cilt,woordlengteratio,aantal_collectieve_voornaamwoorden,aantal_verkleinwoorden,aantal_herhalingen,textcat_elderspeak_score
34,3.730843,74.908909,200.939246,65.17,0.084034,5,3,4,0.142113
35,2.952756,73.815941,103.066537,64.14,0.068966,2,2,4,0.032702
36,3.581527,69.460416,255.741484,64.22,0.065693,7,7,7,0.000798
37,3.959729,71.208296,297.522137,65.18,0.056075,5,1,4,0.022516
38,3.415560,69.129282,195.178867,61.31,0.106796,6,5,3,0.000624


In [22]:
df.corr()

,spraaksnelheid,geluidsniveau,toonhoogte,cilt,woordlengteratio,aantal_collectieve_voornaamwoorden,aantal_verkleinwoorden,aantal_herhalingen,textcat_elderspeak_score
spraaksnelheid,1.000000,0.580838,0.109491,-0.181474,0.085738,0.210640,0.181577,0.203081,-0.156548
geluidsniveau,0.580838,1.000000,-0.141699,0.063167,0.012751,0.080604,0.086111,0.259516,0.151871
toonhoogte,0.109491,-0.141699,1.000000,0.060581,-0.148087,0.353936,-0.157451,0.102996,-0.285911
cilt,-0.181474,0.063167,0.060581,1.000000,-0.816244,0.013630,-0.208198,0.201561,0.166849
woordlengteratio,0.085738,0.012751,-0.148087,-0.816244,1.000000,-0.312774,-0.054573,-0.364718,-0.166749
aantal_collectieve_voornaamwoorden,0.210640,0.080604,0.353936,0.013630,-0.312774,1.000000,0.408538,0.527813,-0.095281
aantal_verkleinwoorden,0.181577,0.086111,-0.157451,-0.208198,-0.054573,0.408538,1.000000,0.655230,0.079258
aantal_herhalingen,0.203081,0.259516,0.102996,0.201561,-0.364718,0.527813,0.655230,1.000000,-0.029017
textcat_elderspeak_score,-0.156548,0.151871,-0.285911,0.166849,-0.166749,-0.095281,0.079258,-0.029017,1.000000


In [23]:
df_r = pd.DataFrame()

for c in df.columns:
    df_r[c + '_res'] = (df[c] > df[c].median())

for c1 in df_r.columns:
    for c2 in df_r.columns:
        if c1 != c2:
            p = st.chi2_contingency(pd.crosstab(df_r[c1], df_r[c2]))[1]
            print('p-waarde = ' + str(p))
            if p < 0.05:
                print('STATISTISCH SIGNIFICANTE CORRELATIE TUSSEN ' + c1 + ' EN ' + c2)
            else:
                print('geen statistisch significante correlatie tussen ' + c1 + ' en ' + c2)

p-waarde = 0.42541679263711385
geen statistisch significante correlatie tussen spraaksnelheid_res en geluidsniveau_res
p-waarde = 0.8759345322834657
geen statistisch significante correlatie tussen spraaksnelheid_res en toonhoogte_res
p-waarde = 1.0
geen statistisch significante correlatie tussen spraaksnelheid_res en cilt_res
p-waarde = 0.2602742916528147
geen statistisch significante correlatie tussen spraaksnelheid_res en woordlengteratio_res
p-waarde = 0.26202180509144596
geen statistisch significante correlatie tussen spraaksnelheid_res en aantal_collectieve_voornaamwoorden_res
p-waarde = 0.43237667839943716
geen statistisch significante correlatie tussen spraaksnelheid_res en aantal_verkleinwoorden_res
p-waarde = 0.2660351186466848
geen statistisch significante correlatie tussen spraaksnelheid_res en aantal_herhalingen_res
p-waarde = 0.6278117988430432
geen statistisch significante correlatie tussen spraaksnelheid_res en textcat_elderspeak_score_res
p-waarde = 0.42541679263711385
